<a href="https://colab.research.google.com/github/akvo/usaid-wssh-tool-3/blob/main/scripts/data-processing-collab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import geopandas as gpd
import requests
from io import StringIO
import os
from sklearn.preprocessing import MinMaxScaler
import ipywidgets as widgets
import matplotlib.pyplot as plt
import folium
from IPython.display import display, clear_output, IFrame
from folium import Choropleth
from IPython.display import IFrame
import requests as req

## Load the dataset

In [ ]:
geojson_uganda = gpd.read_file("https://raw.githubusercontent.com/akvo/usaid-wssh-tool-3/refs/heads/main/data/output/uganda.geojson")

In [ ]:
geojson_uganda.head()

,fid,NAME_2,weighted_drr_raw,weighted_rfr_raw,od_percent_mean,od_population_sum,no_improved_water_premise_population_sum,no_improved_water_premise_population_mean,no_basic_water_sum,no_basic_water_mean,no_basic_sanitation_sum,no_basic_sanitation_mean,weighted_bws_raw,geometry
0,123,PALLISA,0.791179,0.004895,0.086912,21052.918182,298936.862915,0.978612,184739.257935,0.627509,266892.403809,0.889706,0.025201,"MULTIPOLYGON (((33.62633 1.10426, 33.59148 1.1..."
1,124,SERERE,0.762384,0.004082,0.147975,36511.940353,292329.376984,0.967999,170698.406693,0.584093,243208.254532,0.818898,0.010267,"MULTIPOLYGON (((33.58967 1.38705, 33.55708 1.3..."
2,113,KUMI,0.805164,0.001315,0.132785,24206.616264,265178.912201,0.974870,139623.820145,0.539074,223343.025009,0.834607,0.001844,"MULTIPOLYGON (((33.80631 1.31999, 33.75096 1.3..."
3,114,KWEEN,0.791980,0.001497,0.228670,14999.045700,89049.665176,0.987195,38894.484261,0.465947,89492.389008,0.996578,0.004032,"MULTIPOLYGON (((34.5292 1.12092, 34.52641 1.12..."
4,111,KATAKWI,0.807093,0.001964,0.355541,51517.358459,170866.143311,0.990781,92607.968948,0.556583,149845.892517,0.889057,0.003042,"MULTIPOLYGON (((34.20412 2.1235, 34.20843 2.12..."


## Normalize

In [ ]:
# Normalize the relevant columns
# List of columns that need to be normalized (replace with your actual column names)
wash_columns = ['weighted_drr_raw', 'weighted_rfr_raw','weighted_bws_raw' ,'od_percent_mean', 'od_population_sum',
                'no_improved_water_premise_population_sum', 'no_improved_water_premise_population_mean',
               'no_basic_water_sum', 'no_basic_water_mean', 'no_basic_sanitation_sum', "no_basic_sanitation_mean"]  # Example column names

# Initialize the MinMaxScaler
scaler = MinMaxScaler()

# Apply Min-Max scaling to the WASH indicator columns
geojson_uganda[wash_columns] = scaler.fit_transform(geojson_uganda[wash_columns])


In [ ]:
geojson_uganda.describe()

,fid,weighted_drr_raw,weighted_rfr_raw,od_percent_mean,od_population_sum,no_improved_water_premise_population_sum,no_improved_water_premise_population_mean,no_basic_water_sum,no_basic_water_mean,no_basic_sanitation_sum,no_basic_sanitation_mean,weighted_bws_raw
count,135.000000,135.000000,135.000000,135.000000,135.000000,135.000000,135.000000,135.000000,135.000000,135.000000,135.000000,135.000000
mean,68.000000,0.710420,0.286931,0.157034,0.128386,0.186622,0.892966,0.324279,0.686512,0.157802,0.638848,0.082369
std,39.115214,0.190068,0.217255,0.210077,0.146816,0.138720,0.116332,0.211908,0.144732,0.136428,0.259723,0.208460
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,34.500000,0.631669,0.124999,0.038088,0.028952,0.098150,0.859230,0.171559,0.602154,0.079573,0.514152,0.001202
50%,68.000000,0.703171,0.235775,0.085838,0.083134,0.154400,0.918104,0.277452,0.700607,0.130616,0.663503,0.004028
75%,101.500000,0.880669,0.444883,0.181911,0.169081,0.231920,0.964700,0.431583,0.780210,0.186017,0.843850,0.027549
max,135.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


## Rename the columns

In [ ]:
# Rename columns
geojson_uganda.rename(columns={
    'weighted_drr_raw': 'Weighted_drought_risk',
    'weighted_rfr_raw': 'Weighted_riverine_flood_risk',
    'weighted_bws_raw' : 'Weighted_base_water_stress',
    'od_percent_mean': 'Open_defecation_population_mean',
    'od_population_sum': 'Open_defecation_population_sum',
    'no_improved_water_premise_population_sum': 'No_improved_water_on_premises_population_sum',
    'no_improved_water_premise_population_mean': 'No_improved_water_on_premises_population_mean',
    'no_basic_water_sum': 'No_basic_water_population_sum',
    'no_basic_water_mean': 'No_basic_water_population_mean',
    'no_basic_sanitation_sum': 'No_basic_sanitation_population_sum',
    'no_basic_sanitation_mean': 'No_basic_sanitation_population_mean',
}, inplace=True)


## Create an interactive tool for calculating the index

In [ ]:
# Define the indicator options
indicator_options = geojson_uganda.columns.tolist()

# Create widgets for selecting indicators
indicator_selection = widgets.SelectMultiple(
    options=indicator_options,
    description='Indicators',
    disabled=False,
    layout=widgets.Layout(width='400px', height='200px')  # Set custom width and height

)

# Dynamic widgets for core/secondary and directionality selection
core_secondary_widgets = {}
directionality_widgets = {}

# Store selected indicators to accumulate selections
selected_indicators_set = set()

# Main output area to hold all widgets
output_area = widgets.Output()

def create_widgets_for_indicators(indicators):
    core_secondary_widgets.clear()
    directionality_widgets.clear()

    # Output container to hold all widgets
    output_container = []

    for indicator in indicators:
        if indicator not in core_secondary_widgets:
            core_secondary_widgets[indicator] = widgets.Dropdown(
                options=['core', 'secondary'],
                description=f'{indicator} Core/Secondary',
                value='core',  # default value
                layout=widgets.Layout(width='500px'),  # Adjust widget width
                style={'description_width': 'initial'}
            )

        if indicator not in directionality_widgets:
            directionality_widgets[indicator] = widgets.Dropdown(
                options=['positive', 'negative'],
                description=f'{indicator} Direction',
                value='positive',  # default value
                layout=widgets.Layout(width='500px'),  # Adjust widget width
                style={'description_width': 'initial'}
            )

        # Add both widgets (Core/Secondary and Directionality) to a VBox
        output_container.append(widgets.VBox([core_secondary_widgets[indicator], directionality_widgets[indicator]]))

    return widgets.VBox(output_container)


# Weights: Core = 0.7, Secondary = 0.3
weight_mapping = {'core': 0.7, 'secondary': 0.3}

In [ ]:
# Create a button to update indicator widgets dynamically
update_button = widgets.Button(description="Update Indicators")

#Create a delete button
delete_button = widgets.Button(description="Delete Selected Indicator")

# Create a button to calculate the index
calculate_button = widgets.Button(description="Calculate Index")

In [ ]:
# Create output areas to control what gets cleared
output_area = widgets.Output()  # For displaying buttons and widgets
map_area = widgets.Output()     # For displaying the map

In [ ]:
def apply_directionality(data, indicators, directionality):
    directed_data = {}
    for idx, indicator in enumerate(indicators):
        if directionality[indicator] == 'positive':
            directed_data[indicator] = data[indicator]  # Positive direction, keep the value as is
        else:
            directed_data[indicator] = 1 - data[indicator]  # Negative direction, invert the value (1 - value)
    return directed_data

In [ ]:
# Function to calculate the weighted index and show on the map
def calculate_index(indicators, core_secondary, directionality):
    directed_data = apply_directionality(geojson_uganda, indicators, directionality)
    geojson_uganda['index'] = 0

    for indicator in indicators:
        weight = weight_mapping[core_secondary[indicator]]
        geojson_uganda['index'] += directed_data[indicator] * weight

    # Clear the map area to prevent stacking maps
    with map_area:
        clear_output(wait=True)

    # Create a choropleth map with folium
    m = folium.Map(location=[1.3733, 32.2903], zoom_start=7)

    Choropleth(
        geo_data=geojson_uganda.to_json(),
        data=geojson_uganda,
        columns=["NAME_2", "index"],
        key_on="feature.properties.NAME_2",
        fill_color="YlGnBu",
        fill_opacity=0.7,
        line_opacity=0.2,
        legend_name="WASH Index"
    ).add_to(m)

    # Add a tooltip to display the index score on hover
    folium.GeoJson(
        geojson_uganda,
        style_function=lambda x: {'fillColor': '#ffffff00', 'color': '#00000000', 'weight': 0},
        tooltip=folium.GeoJsonTooltip(fields=["NAME_2", "index"],
                                      aliases=["Region: ", "Index Score: "],
                                      localize=True)
    ).add_to(m)

    # Add a custom legend with "Low priority" and "High priority" labels
    legend_html = '''
     <div style="position: fixed;
     bottom: 50px; left: 50px; width: 150px; height: 90px;
     background-color: white; border:2px solid grey; z-index:9999; font-size:14px;
     padding: 10px;">
     <b>Priority Legend</b><br>
     <i style="background: #ffffcc; width: 20px; height: 10px; display: inline-block;"></i> Low priority<br>
     <i style="background: #08306b; width: 20px; height: 10px; display: inline-block;"></i> High priority<br>
     </div>
     '''
    m.get_root().html.add_child(folium.Element(legend_html))

    display(m)
    # Save the map to an HTML file and display it in an IFrame
    #map_file = 'index_map.html'
    #m.save(map_file)
    #display(IFrame(map_file, width=700, height=500))

In [ ]:
# Function to handle the button click event for updating widgets
def on_update_button_click(b):
    # Append selected indicators to the set to avoid removing previous selections
    selected_indicators_set.update(list(indicator_selection.value))

    # Clear the output area (not map) and create widgets for each selected indicator
    with output_area:
        clear_output(wait=True)
        # Display indicator list, update button, and delete button
        display(indicator_selection, update_button, delete_button, calculate_button)
        # Display the Core/Secondary and Directionality dropdowns
        display(create_widgets_for_indicators(selected_indicators_set))
        # Display the calculate index button
        #display(calculate_button)


    # Function to handle the button click event for deleting widgets
def on_delete_button_click(b):
    # Remove selected indicators from the set
    selected_indicators_set.difference_update(list(indicator_selection.value))

    # Clear the output area (not map) and update widgets after deletion
    with output_area:
        clear_output(wait=True)
        # Display indicator list, update button, and delete button
        display(indicator_selection, update_button, delete_button)
        # Display the Core/Secondary and Directionality dropdowns
        display(create_widgets_for_indicators(selected_indicators_set))
        # Display the calculate index button
        display(calculate_button)

# Function to display widgets for the selected indicators
def display_widgets(indicators):
    clear_output(wait=True)  # Clear previous output
    create_widgets_for_indicators(indicators)

    display(indicator_selection, update_button, delete_button)  # Re-display the selection and update button

    for indicator in indicators:
        display(core_secondary_widgets[indicator])
        display(directionality_widgets[indicator])

    display(calculate_button)  # Show the calculate button after displaying all the widgets

# Function to handle the button click event for calculating the index
def on_calculate_button_click(b):
    selected_indicators = list(indicator_selection.value)

    # Collect core/secondary and directionality choices
    core_sec = {indicator: core_secondary_widgets[indicator].value for indicator in selected_indicators}
    directionality = {indicator: directionality_widgets[indicator].value for indicator in selected_indicators}

    calculate_index(selected_indicators, core_sec, directionality)

In [ ]:
# Attach the button click events to the handler functions
update_button.on_click(on_update_button_click)
delete_button.on_click(on_delete_button_click)
calculate_button.on_click(on_calculate_button_click)

In [ ]:
# Display the initial selection widget and buttons
with output_area:
    display(indicator_selection, update_button, delete_button, calculate_button)

# Display both the control widgets and the map area
display(output_area, map_area)

Output()

Output()